In [5]:
from ultralytics import YOLO
import pandas as pd
import json
import csv

model = YOLO('yolov8n.pt')
# train_boxes = pd.read_csv('oidv6-train-annotations-bbox.csv')


In [6]:
def get_food_labels():
    food_label = '/m/02wbm'
    with open('bbox_labels_600_hierarchy.json', 'r') as file:
        bbox_labels = json.load(file)

    def find_all_food(food_dict, add_all=False):
        labels = []
        if food_dict['LabelName'] == food_label or add_all:
            add_all = True
            labels.append(food_dict['LabelName'])

        if 'Subcategory' in food_dict:
            for subcategory in food_dict['Subcategory']:
                labels.extend(find_all_food(subcategory, add_all))

        return labels

    return find_all_food(bbox_labels)

food_labels = get_food_labels()


In [7]:
image_train_dataset = []
with open('oidv6-train-annotations-bbox.csv', 'r') as f:
    reader = csv.reader(f)
    for record in reader:
        label = record[2]
        if label in food_labels:
            image_train_dataset.append((record[0], label, record[4], record[5], record[6], record[7]))
        # image_id = record[0]
        # x_min = record[4]
        # x_max = record[5]
        # y_min = record[6]
        # y_max = record[7]
        # if label in food_labels:
        #     image_train_dataset.append((image_id, label, x_min, x_max, y_min, y_max))

KeyboardInterrupt: 